<a href="https://colab.research.google.com/github/Juapatral/webscrapers/blob/main/fincaraiz_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
print(
"""
#-------------------------------------------------------------------------
# Author: Juan Pablo Trujillo Alviz 
# github: juapatral
# CD: 2020-09-24 
# LUD: 2020-09-24
# Description: webscrapping of fincaraiz website
# Use: modify weblink and file_name
#
#
# v3
# Modification:
# Description:
#-------------------------------------------------------------------------
"""
)


#-------------------------------------------------------------------------
# Author: Juan Pablo Trujillo Alviz 
# github: juapatral
# CD: 2020-09-24 
# LUD: 2020-09-24
# Description: webscrapping of fincaraiz website
# Use: modify weblink and file_name
#
#
# v3
# Modification:
# Description:
#-------------------------------------------------------------------------



In [ ]:
### webscrapping fincaraiz

## importar librerias

import numpy as np
import pandas as pd
import re
import datetime as dt
import glob
import requests
from bs4 import BeautifulSoup
import urllib3
import time
from google.colab import files

In [ ]:
## crear funciones propias

def diccionario(nuevo_texto):
    """Crea un diccionario con una lista de tuplas llave valor, 
    si no tiene valor queda nulo
    """
    
    # crear diciconario vacio
    dicc = {}

    # iterar sobre la lista
    for tupla in nuevo_texto:
        # crear variable
        try:
            tupla2 = tupla[1] if tupla[1] != "" else None
            dicc[tupla[0]] = tupla2
        except:
            pass

    return dicc

In [ ]:
## parametros fijos

# date time
now = dt.datetime.now()

# conteo de paginas y maximo numero de paginas a buscar
number_page, max_number_page = 1, 7000

# pagina inicial a buscar
weblink = "https://www.fincaraiz.com.co/Vivienda/venta/"
path = "" # cambiar si quieren alguna ruta de su drive, por ejemplo
#ubicacion = weblink.split("/")[5].lower()
tipo_inmueble = weblink.split("/")[3].lower()
tipo_negocio = weblink.split("/")[4].lower()
fecha = now.year * 10000 + now.month * 100 + now.day
file_name = (
    #"fincaraiz-{}-{}_{}-{}.csv"
    "fincaraiz-{}-{}_{}.csv"
    #.format(ubicacion, tipo_inmueble, tipo_negocio, fecha)
    .format(tipo_inmueble, tipo_negocio, fecha)
)

# desahilitar advertencias
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# show time
print("EXTRACCIÓN DE: {}".format(file_name))
print("Empieza la extracción a las {}".format(now))

EXTRACCIÓN DE: fincaraiz-vivienda-venta_20210413.csv
Empieza la extracción a las 2021-04-13 00:57:08.779847


In [ ]:
### ejecucion

## parte 1 extraer urls de las paginas

# encontrar archivo de urls
urls_df_list = glob.glob("./*urls*")


if urls_df_list == []:

    # listas vacias de url
    urls = []

    # crear loop que vaya por cada pagina solicitada
    while number_page <= max_number_page:
        number_page += 1

        # hacer el request
        try:
            r = requests.get(weblink, timeout=(15, 15), verify=False)
        except:
            break

        # si la conexion no es exitosa
        if r.status_code != 200:
            break

        # convertir el html en texto
        content = r.content
        soup = BeautifulSoup(content, "html.parser")


        # encontrar pagina siguiente
        try:
            weblink = soup.find("link", {"rel":"next"})["href"]    
        except:
            break

        # encontrar bloques de ventas
        divAdverts = soup.find("div", {"id":"divAdverts"})

        # encontrar cada bloque de venta
        ul = divAdverts.find_all("ul")

        # extraer las urls de cada bloque
        for u in ul:
            ads = u.find("div", {"class":"AdvertisingContainer"})

            # si es un anuncio se omite
            if ads is not None:
                continue
            
            # extraer las url
            try:
                url = u.find_all("a", href=True)
                full_url = "https://www.fincaraiz.com.co" + str(url[0]['href'])

            except:
                full_url = None
            
            # adicionar al listado de urls
            urls.append(full_url)

    # informar
    print("Se extrajo {} páginas de información".format(number_page))

    # limpiar urls nulas 
    urls = [url for url in urls if url is not None]
    urls = [*set(urls)]
    # informar
    print("Y se encontraron {} urls de inmuebles".format(len(urls)))

    # crear dataframe
    urls_df = pd.DataFrame(urls)
    urls_df.to_csv(path + "urls_" + file_name, index=False)
    files.download(path + "urls_" + file_name)
else:
    urls_df = pd.read_csv(urls_df_list[0])
    urls = [*urls_df["0"]]

In [ ]:
## parte 2 extraer informacion de las urls

# funcion de particiones
def particiones(numb, part=50000):
    return numb//part + (1 if numb%part != 0 else 0)

def extraer_info_urls(urls, iteracion=0):

    # crear lista diccionarios vacia
    lista_dicc = []
    
    # request por cada url
    for i, url in enumerate(urls):

        # mostrar avance
        texto = (
            "\rProcesado: " + str(i+1) + "/" + str(len(urls)) + " " + 
            str(100*(i+1)/len(urls)) + "%"
        )
        if i + 1 == len(urls):
            texto += "\r"
        print(texto, end="")    
        time.sleep(0.1)
        
        # hacer el request
        try:
            inm_r = requests.get(url, verify=False)
        except:
            continue
        
        if inm_r.status_code != 200:
            continue
            
        inm_content = inm_r.content
        inm_soup = BeautifulSoup(inm_content, "html.parser")    

        # encontrar json con informacion
        scripts = inm_soup.find_all("script", {"type":"text/javascript"})
        texto = []

        # si el request falla
        if scripts == []:
            continue

        # extraer el json
        for s in scripts:
            texto = re.findall("var sfAdvert = {(.*)}", s.text)
            if texto != []:
                break
        
        # si no encuentra informacion del inmueble
        if texto == []:
            continue

        # convertir el json en un diccionario
        nuevo_texto = texto[0].split("\", ")
        nuevo_texto2 = [t.replace("\"","").split(" : ") for t in nuevo_texto]
        dicc = diccionario(nuevo_texto2)

        # adicionar a la lista de diccionarios
        lista_dicc.append(dicc)

    ## parte 3 creacion del dataframe

    # convertir cada diccionario en una lista de dataframe
    lista_df = [pd.DataFrame(dic, index=[0]) for dic in lista_dicc]

    # concatenar dataframe
    df = pd.concat(lista_df).reset_index(drop=True)

    # crear columnas faltantes
    df["extraction_year"] = now.year
    df["extraction_month"] = now.month
    df["extraction_day"] = now.day

    # eliminar duplicados
    df.drop_duplicates(inplace=True)

    # guardar dataframe
    df.to_csv(path + file_name.replace(".csv", "-parte{}.csv").format(iteracion), index=False)
    files.download(path + file_name.replace(".csv", "-parte{}.csv").format(iteracion))

    # tiempo
    print("Se demoró (HH:MM:SS): ", dt.datetime.now()-now)

In [ ]:
# limpiar urls nulas 
urls = [url for url in urls if url is not None]

# datos
partes = 30000
inicio = 6

# crear particiones
parts = particiones(len(urls), part=partes)
print("Se extraerán {} particiones".format(parts))

for part in range(inicio, parts):
    print("Inicia la partición {}".format(part+1))
    rango_i, rango_f = partes*part, partes*(part+1)
    rango_f = rango_f if rango_f < len(urls) else len(urls)
    urls_partidas = urls[rango_i:rango_f]
    extraer_info_urls(urls_partidas, part)

Se extraerán 7 particiones
Inicia la partición 7


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Se demoró (HH:MM:SS):  2:41:42.772703


In [ ]:

## parte 3 creacion del dataframe

# convertir cada diccionario en una lista de dataframe
lista_df = [pd.DataFrame(dic, index=[0]) for dic in lista_dicc]

# concatenar dataframe
df = pd.concat(lista_df).reset_index(drop=True)

# crear columnas faltantes
df["extraction_year"] = now.year
df["extraction_month"] = now.month
df["extraction_day"] = now.day

# eliminar duplicados
df.drop_duplicates(inplace=True)

# guardar dataframe
df.to_csv(path + file_name, index=False)
files.download(path + file_name) 

# tiempo
print("Se demoró (HH:MM:SS): ", dt.datetime.now()-now)

In [ ]:
 urls = pd.read_csv("/content/urls_fincaraiz-cundinamarca-vivienda_venta-20200930.csv")
urls = [*urls["0"]]

In [ ]:
# -*- coding: utf-8 -*-
"""fincaraiz_webscrapping.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1HtkSUSD5gBVdPHUEOgMo6si9yihmGCT9
"""

# -*- coding: utf-8 -*-
"""
#-------------------------------------------------------------------------
# Author: Juan Pablo Trujillo Alviz 
# github: juapatral
# CD: 2020-09-24 
# LUD: 2020-09-24
# Description: webscrapping of fincaraiz website
#
#
# example of use:
#>>>ciudad = webscrapping(
#               "path_of_directory", 
#               "www.fincaraiz.com.co/Vivienda/venta/medellin"
#)
#-------------------------------------------------------------------------
"""

### webscrapping fincaraiz

## importar librerias

import numpy as np
import pandas as pd
import re
import datetime as dt
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import os

## crear funciones propias

def diccionario(nuevo_texto):
    """Crea un diccionario con una lista de tuplas llave valor, 
    si no tiene valor queda nulo
    """
    
    # crear diciconario vacio
    dicc = {}

    # iterar sobre la lista
    for tupla in nuevo_texto:
        # crear variable
        tupla2 = tupla[1] if tupla[1]!="" else None
        dicc[tupla[0]] = tupla2
    return dicc

## parametros fijos

def webscrapping(data_path, weblink):
    
    # date time
    now = dt.datetime.now()

    # conteo de paginas y maximo numero de paginas a buscar
    number_page, max_number_page = 1, 10

    # listas vacias de url y diccionario de datos
    urls, lista_dicc = [], []

    # pagina inicial a buscar
    #weblink = "https://www.fincaraiz.com.co/arriendos/"

    # nombrar el weblink
    ubicacion = weblink.split("/")[5].lower()
    tipo_inmueble = weblink.split("/")[3].lower()
    tipo_negocio = weblink.split("/")[4].lower()
    fecha = now.year * 10000 + now.month * 100 + now.day
    file_name = (
        "fincaraiz-{}-{}_{}-{}.csv"
        .format(ubicacion, tipo_inmueble, tipo_negocio, fecha)
    )

    # desahilitar advertencias
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    ### ejecucion

    ## parte 1 extraer urls de las paginas

    # crear loop que vaya por cada pagina solicitada
    while number_page <= max_number_page:
        number_page += 1

        # hacer el request
        r = requests.get(weblink, verify=False)
        content = r.content
        soup = BeautifulSoup(content, "html.parser")


        # encontrar pagina siguiente
        weblink = soup.find("link", {"rel":"next"})["href"]    

        # encontrar bloques de ventas
        divAdverts = soup.find("div", {"id":"divAdverts"})

        # encontrar cada bloque de venta
        ul = divAdverts.find_all("ul")

        # extraer las urls de cada bloque
        for u in ul:
            ads = u.find("div", {"class":"AdvertisingContainer"})

            # si es un anuncio se omite
            if ads is not None:
                continue
            
            # extraer las url
            try:
                url = u.find_all("a", href=True)
                full_url = "https://www.fincaraiz.com.co" + str(url[0]['href'])

            except:
                full_url = None
            
            # adicionar al listado de urls
            full_url = full_url if full_url.count("http") <= 1 else None
            urls.append(full_url)

    ## parte 2 extraer informacion de las urls
    
    # limpiar urls nulas 
    urls = [url for url in urls if url is not None]

    # request por cada url
    for i, url in enumerate(urls):

        # mostrar avance
        texto = (
            "\rProcesado: " + str(i+1) + "/" + str(len(urls)) + " " + 
            str(100*(i+1)/len(urls)) + "%"
        )
        if i + 1 == len(urls):
            texto += "\r"
        print(texto, end="")    
        time.sleep(0.1)
        
        # hacer el request
        inm_r = requests.get(url, verify=False)
        inm_content = inm_r.content
        inm_soup = BeautifulSoup(inm_content, "html.parser")    

        # encontrar json con informacion
        scripts = inm_soup.find_all("script", {"type":"text/javascript"})
        texto = []

        # si el request falla
        if scripts == []:
            continue

        # extraer el json
        for s in scripts:
            texto = re.findall("var sfAdvert = {(.*)}", s.text)
            if texto != []:
                break
        
        # si no encuentra informacion del inmueble
        if texto == []:
            continue

        # convertir el json en un diccionario
        nuevo_texto = texto[0].split("\", ")
        nuevo_texto2 = [t.replace("\"","").split(" : ") for t in nuevo_texto]
        dicc = diccionario(nuevo_texto2)

        # adicionar a la lista de diccionarios
        lista_dicc.append(dicc)

    ## parte 3 creacion del dataframe

    # convertir cada diccionario en una lista de dataframe
    lista_df = [pd.DataFrame(dic, index=[0]) for dic in lista_dicc]

    # concatenar dataframe
    df = pd.concat(lista_df).reset_index(drop=True)

    # crear columnas faltantes
    df["extraction_year"] = now.year
    df["extraction_month"] = now.month
    df["extraction_day"] = now.day

    # eliminar duplicados
    df.drop_duplicates(inplace=True)

    # tiempo
    print("Se demoró (HH:MM:SS): ", dt.datetime.now()-now)

    # guardar dataframe
    df.to_csv(data_path + file_name, index=False)
    return df

In [ ]:
data_path = ""
weblink = "https://www.fincaraiz.com.co/Vivienda/venta/fusagasuga/"

In [ ]:
site = "https://www.fincaraiz.com.co/proyecto-parque-la-colina--coburgo/fusagasuga/proyecto-nuevo-det-1140183.aspx?itemid=5245095&pag=1&pos=3&IsProjectWithOutProduct=True"

In [ ]:
import requests

In [ ]:
r = requests.get(site)
content = r.content
soup = BeautifulSoup(content, "html.parser")

In [ ]:
soup